<a href="https://colab.research.google.com/github/xinruili07/Fashion-MNIST/blob/master/FashionMnist_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import json

!pip install -q kaggle
!mkdir /content/.kaggle
!mkdir ~/.kaggle
token = {"username":"xinruili","key":"5971264a8ca5983c4f4864ea88dfa8b5"}
with open("/content/.kaggle/kaggle.json", "w") as file:
    json.dump(token, file)
!chmod 600 /content/.kaggle/kaggle.json
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
!kaggle config set -n path -v{/content}
!kaggle competitions download -c mais202-fall2019
!apt install unzip

!unzip /content/{/content}/competitions/mais202-fall2019/test_images.npy.zip -d test_images
!unzip /content/{/content}/competitions/mais202-fall2019/train_images.npy.zip -d train_images

- path is now set to: {/content}
  0% 0.00/127 [00:00<?, ?B/s]
100% 127/127 [00:00<00:00, 291kB/s]
 43% 9.00M/20.7M [00:00<00:00, 35.8MB/s]
100% 20.7M/20.7M [00:00<00:00, 59.5MB/s]
  0% 0.00/429k [00:00<?, ?B/s]
100% 429k/429k [00:00<00:00, 156MB/s]
  0% 0.00/165k [00:00<?, ?B/s]
100% 165k/165k [00:00<00:00, 160MB/s]
 95% 49.0M/51.7M [00:03<00:00, 8.76MB/s]
100% 51.7M/51.7M [00:03<00:00, 17.7MB/s]
Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-21ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 28 not upgraded.
Archive:  /content/{/content}/competitions/mais202-fall2019/test_images.npy.zip
  inflating: test_images/test_images.npy  
Archive:  /content/{/content}/competitions/mais202-fall2019/train_images.npy.zip
  inflating: train_images/train_images.npy  


In [0]:
#vanilla CNN
from keras.datasets import fashion_mnist
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
from keras import backend as K

Using TensorFlow backend.


In [0]:
from sklearn.model_selection import train_test_split
X_train = np.load('/content/train_images/train_images.npy')

X_train = X_train.astype("float32") / 255.0
X_train = X_train.reshape((X_train.shape[0], 28, 28, 1))
print(X_train.shape)

(50000, 28, 28, 1)


In [0]:
from keras.utils import to_categorical

data = pd.read_csv("/content/{/content}/competitions/mais202-fall2019/train_labels.csv", header=0)
y_train = list(data.label)
y_train = to_categorical(y_train)
print(len(y_train))

50000


In [0]:
from sklearn.utils import shuffle

X_train, y_train = shuffle(X_train, y_train)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.20)

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import time

NAME = "kaggle-competition-{}".format(int(time.time()))

# tensorboard --logdir='DIRNAME' to view
tensorboard = TensorBoard(log_dir='logs\{}'.format(NAME))

In [0]:
from sklearn.model_selection import train_test_split
print(len(X_train))
print(len(X_valid))

32000
8000


In [0]:
model = Sequential()

model.add(Conv2D(32, (5, 5), padding="same", input_shape=[28, 28, 1], kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (5, 5), padding="same"))
model.add(MaxPooling2D((2,2)))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

model.summary()

Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_39 (Conv2D)           (None, 28, 28, 32)        832       
_________________________________________________________________
activation_19 (Activation)   (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_33 (Dropout)         (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 14, 14, 64)        51264     
_________________________________________________________________
max_pooling2d_31 (MaxPooling (None, 7, 7, 64)          0         
_________________________________________________________________
activation_20 (Activation)   (None, 7, 7, 64)        

In [0]:
import keras
model.compile(keras.optimizers.Adam(5e-4), loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
history = model.fit(X_train, y_train, epochs = 25, shuffle=True, validation_data=(X_valid, y_valid), verbose=1, batch_size=64)

Train on 40000 samples, validate on 10000 samples
Epoch 1/25
40000/40000 [==============================] - 10s 252us/step - loss: 0.6462 - acc: 0.7597 - val_loss: 0.4974 - val_acc: 0.8073
Epoch 2/25
40000/40000 [==============================] - 10s 247us/step - loss: 0.5124 - acc: 0.8121 - val_loss: 0.4236 - val_acc: 0.8419
Epoch 3/25
40000/40000 [==============================] - 10s 245us/step - loss: 0.4584 - acc: 0.8323 - val_loss: 0.4145 - val_acc: 0.8397
Epoch 4/25
40000/40000 [==============================] - 10s 251us/step - loss: 0.4265 - acc: 0.8424 - val_loss: 0.3817 - val_acc: 0.8522
Epoch 5/25
40000/40000 [==============================] - 10s 249us/step - loss: 0.3985 - acc: 0.8530 - val_loss: 0.3712 - val_acc: 0.8587
Epoch 6/25
40000/40000 [==============================] - 10s 252us/step - loss: 0.3800 - acc: 0.8600 - val_loss: 0.3537 - val_acc: 0.8653
Epoch 7/25
40000/40000 [==============================] - 10s 245us/step - loss: 0.3608 - acc: 0.8664 - val_loss: 0.

In [0]:
from sklearn.metrics import confusion_matrix

X_test = np.load('/content/test_images/test_images.npy').squeeze()
X_test = X_test.astype("float32") / 255.0
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)
print(X_test.shape)
y_pred = model.predict_classes(X_test)

df_test = pd.read_csv('/content/{/content}/competitions/mais202-fall2019/sample_submission.csv')
df_test['label'] = y_pred
df_test.to_csv('submission.csv', index=False)

(20000, 28, 28, 1)
[[1871    0   17   52    3    0  259    0    3    0]
 [   1 1955    1   27    1    0    5    0    1    0]
 [  28    2 1683    2   85    1  123    0    8    0]
 [  79   14   16 1863   97    2   83    0    4    0]
 [   4    4  278   32 1802    0  247    0    1    0]
 [   3    0    2    0    1 1890    1   23    2    5]
 [  96    1  104   17   44    0 1120    0   15    0]
 [   0    0    0    0    0   65    0 1930    3   39]
 [   8    0   10    6    4    3   15    4 1931    1]
 [   0    1    0    0    0   13    0   58    0 1931]]
